#DEMO carga incremental

## Dividir los dataframes crudos para demostracion

In [1]:
import datetime
import numpy as np
import pandas as pd

Hemos dividido el csv crudo de "chekin" en particiones para llevar a cabo la prueba.

In [ ]:
df_checkin = pd.read_json("/content/drive/MyDrive/yelp/Copia de checkin.json", lines=True)
# Obtener la fecha actual
fecha_actual = datetime.date.today().strftime("%Y%m%d")  # Formato AAAAMMDD

batch_size = len(df_checkin) // 2
# Dividir el DataFrame en lotes más pequeños
df_batches = [df_checkin[i:i+batch_size] for i in range(0, len(df_checkin), batch_size)]

# Guardar cada lote en un archivo CSV con la fecha en el nombre
for i, batch in enumerate(df_batches):
    file_name = f'/content/drive/MyDrive/prueba_carga_inc/yelp_data_cruda/checkin_{fecha_actual}_{i+1}.csv'
    batch.to_csv(file_name, index=False)


Realizamos lo mismo con business

In [5]:
# Leer el archivo pickle y seleccionar solo las columnas deseadas
columns_to_keep = ['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count']
df_business = pd.read_csv('/content/drive/MyDrive/Otros/yelp_business_florida_final.csv')[columns_to_keep]

# Obtener la fecha actual
fecha_actual = datetime.date.today().strftime("%Y%m%d")  # Formato AAAAMMDD

batch_size = len(df_business) // 2
# Dividir el DataFrame en lotes más pequeños
df_batches = [df_business[i:i+batch_size] for i in range(0, len(df_business), batch_size)]

# Guardar cada lote en un archivo CSV con la fecha y número de lote en el nombre
for i, batch in enumerate(df_batches):
    file_name = f'/content/drive/MyDrive/prueba_carga_inc/yelp_data_cruda/business_{fecha_actual}_{i+1}.csv'
    batch.to_csv(file_name, index=False)




<ipython-input-5-03929f79e3b4>:3: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_business = pd.read_csv('/content/drive/MyDrive/Otros/yelp_business_florida_final.csv')[columns_to_keep]


##Funciones para la carga incremental

Creamos funcion para buscar en la carpeta de los datos crudos

In [6]:
import os

def get_latest_file(folder_path, prefix):
    # Listar los archivos en la carpeta
    files_in_folder = os.listdir(folder_path)

    latest_file = None

    # Filtrar archivos con el prefijo proporcionado
    filtered_files = [file for file in files_in_folder if file.startswith(prefix) and file.endswith('.csv')]
    if filtered_files:
        latest_file = max(filtered_files, key=lambda x: x.split('_')[1])

    return latest_file

**Obtiene los archivos con la fecha mas reciente**


In [7]:
print(get_latest_file('/content/drive/MyDrive/prueba_carga_inc/yelp_data_cruda', 'checkin'))
print(get_latest_file('/content/drive/MyDrive/prueba_carga_inc/yelp_data_cruda', 'business'))

checkin_20230830.csv
business_20230830.csv


Realizamos funcion para etl de busines de muestra

In [21]:
def process_yelp_data_business():
    file_path = f"/content/drive/MyDrive/prueba_carga_inc/yelp_data_cruda/{get_latest_file('/content/drive/MyDrive/prueba_carga_inc/yelp_data_cruda', 'business')}"
    df = pd.read_csv(file_path)
    return df

In [28]:
process_yelp_data_business().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8986 entries, 0 to 8985
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   8986 non-null   object 
 1   name          8986 non-null   object 
 2   address       8719 non-null   object 
 3   city          8986 non-null   object 
 4   state         8986 non-null   object 
 5   postal_code   8986 non-null   float64
 6   latitude      8986 non-null   float64
 7   longitude     8986 non-null   float64
 8   stars         8986 non-null   float64
 9   review_count  8986 non-null   int64  
dtypes: float64(4), int64(1), object(5)
memory usage: 702.2+ KB


Creamos funcion de muestra realizando etl de checkin

In [8]:
def process_yelp_data_checkin():
    yelp_checkin_crudo = pd.read_csv(f"/content/drive/MyDrive/prueba_carga_inc/yelp_data_cruda/{get_latest_file('/content/drive/MyDrive/prueba_carga_inc/yelp_data_cruda', 'checkin')}")
    yelp_business_crudo = pd.read_csv(f"/content/drive/MyDrive/prueba_carga_inc/yelp_data_cruda/{get_latest_file('/content/drive/MyDrive/prueba_carga_inc/yelp_data_cruda', 'business')}")

    yelp_checkin_ETL = yelp_checkin_crudo.merge(yelp_business_crudo, on='business_id', how='inner')

    # Dividir las fechas y horas en la columna 'date' y contar los elementos resultantes
    yelp_checkin_ETL['checkin_count'] = yelp_checkin_ETL['date'].apply(lambda x: len(x.split(', ')))

    # Filtrar el DataFrame resultante
    yelp_checkin_ETL_filtered = yelp_checkin_ETL[['business_id', 'checkin_count']]

    return yelp_checkin_ETL_filtered

In [ ]:
process_yelp_data_checkin()

<ipython-input-96-2eb877907976>:3: DtypeWarning: Columns (14,15,16,17,18,19,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  yelp_business_crudo = pd.read_csv(f"/content/drive/MyDrive/prueba_carga_inc/yelp_data_cruda/{get_latest_file('/content/drive/MyDrive/prueba_carga_inc/yelp_data_cruda', 'business')}")


,business_id,checkin_count
0,---kPU91CF4Lq2-WlRu9Lw,11
1,--30_8IhuyMHbSOcNWd6DQ,2
2,--7PUidqRWpRSpXebiyxTg,10
3,--8IbOsAAxjKRoYsBFL-PA,32
4,--FWWsIwxRwuw9vIMImcQg,7
...,...,...
32880,V4EO-z6APccp3VctHYOZug,73
32881,V4Kkuej274hGsnd3QM2f_Q,6
32882,V4Sv9A8VZUmEBEZIUaQzcw,10
32883,V4THIJk9uhoxL7f06fK5VA,15


Realizamos el proceso para crear tabla con los datos base y la carga incremental


In [11]:
from google.colab import auth
auth.authenticate_user()
from google.cloud import bigquery
import pandas as pd
import pandas_gbq

Funcion para crear tabla y insertar datos para realizar la primera carga

In [13]:
def load_dataframe_to_bigquery(job_config, df, table_name):
    # Crea una instancia del cliente de BigQuery
    client = bigquery.Client(project="finalprojectprototype-397114")

    # Define los detalles de la tabla a la que deseas cargar los datos
    table_id = f"finalprojectprototype-397114.yelp.{table_name}"  # Cambia a tu ID de proyecto y conjunto de datos

    # Carga el DataFrame en la tabla de BigQuery
    job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
    job.result()  # Espera a que se complete el proceso de carga

    print(f"Datos del DataFrame insertados en la tabla: {table_id}")


Creamos la tabla para business

In [37]:
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("business_id", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("address", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("city", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("state", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("postal_code", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("latitude", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("longitude", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("stars", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("review_count", "INTEGER", mode="NULLABLE")
    ]
)
load_dataframe_to_bigquery(job_config, process_yelp_data_business(),'business')


Datos del DataFrame insertados en la tabla: finalprojectprototype-397114.yelp.business


creamos la tabla para checkin

In [38]:
# Definir el job_config para checkin
job_config = bigquery.LoadJobConfig(schema=[
    bigquery.SchemaField("business_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("checkin_count", "INTEGER", mode="NULLABLE")
])


# Llamar a la función para cargar el DataFrame en BigQuery
load_dataframe_to_bigquery(job_config, process_yelp_data_checkin(),'checkin')

Datos del DataFrame insertados en la tabla: finalprojectprototype-397114.yelp.checkin


Funcion para insertar datos de manera incremental en latabla ya creada

In [31]:
def insert_dataframe_into_bigquery(dataframe, project_id, dataset_id, table_id):
    dataframe.to_gbq(destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists="append")



insertamos los datos nuevos de checkin

In [33]:
insert_dataframe_into_bigquery(process_yelp_data_checkin(),"finalprojectprototype-397114",'yelp','checkin')

100%|██████████| 1/1 [00:00<00:00, 722.78it/s]


insertamos los datos nuevos de checkin

In [35]:
insert_dataframe_into_bigquery(process_yelp_data_business(),"finalprojectprototype-397114",'yelp','business')

100%|██████████| 1/1 [00:00<00:00, 1561.54it/s]


Funcion para contar la cantidad de fila de la tabla prueba de big query



In [36]:
def count_rows_in_bigquery_table(project_id, dataset_id, table_id):
    client = bigquery.Client(project=project_id)
    query = f"SELECT COUNT(*) as row_count FROM `{project_id}.{dataset_id}.{table_id}`"
    query_job = client.query(query)
    result = query_job.result()
    row_count = next(result).row_count
    return row_count

project_id = "finalprojectprototype-397114"
dataset_id = "yelp"
table_id = "checkin"

row_count = count_rows_in_bigquery_table(project_id, dataset_id, table_id)
print(f"La tabla tiene {row_count} filas.")


La tabla tiene 7714 filas.
